#Install required dependency and SDK

In [ ]:
!pip install --quiet google-cloud-bigquery google-cloud-aiplatform pandas

In [ ]:
from google.cloud import bigquery
from vertexai.preview.language_models import ChatModel
import vertexai
import pandas as pd

#Configuring Environment variable

In [ ]:

PROJECT_ID = "qwiklabs-gcp-02-4c9c7fb5e8ec"
LOCATION = "us-central1"
BQ_DATASET = "alaska_dept_of_snow"
TABLE_RAW = "faq_data"
TABLE_EMBEDDED = "faq_data_embedded"
EMBED_MODEL = "faq_embeddings"
TABLE_ID = f"{PROJECT_ID}.{BQ_DATASET}.{TABLE_EMBEDDED}"
RAW_TABLE_ID = f"{PROJECT_ID}.{BQ_DATASET}.{TABLE_RAW}"
EMBED_MODEL_ID = f"{PROJECT_ID}.{BQ_DATASET}.{EMBED_MODEL}"
GS_URI = "gs://labs.roitraining.com/alaska-dept-of-snow/alaska-dept-of-snow-faqs.csv"


In [ ]:
vertexai.init(project=PROJECT_ID, location=LOCATION)
bq_client = bigquery.Client(project=PROJECT_ID)

#Create the Dataset as alaksa_dept_of_snow

In [ ]:
# Create the dataset if it doesn't exist
from google.cloud import exceptions
dataset_id = f"{bq_client.project}.{BQ_DATASET}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = LOCATION

try:
    bq_client.get_dataset(dataset_id)  # Raises NotFound if the dataset does not exist
    print(f"Dataset {dataset_id} already exists")
except exceptions.NotFound: # Changed NotFound to exceptions.NotFound
    dataset = bq_client.create_dataset(dataset, timeout=30)  # Make an API request.
    print(f"Created dataset {dataset_id}")

Dataset qwiklabs-gcp-02-4c9c7fb5e8ec.alaska_dept_of_snow already exists


#Load the data to Table

In [ ]:
uri = GS_URI
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition="WRITE_TRUNCATE"
)
# Load the csv file data into the table named faq_data in the dataset we created alaska_dept_of_snow
load_job = bq_client.load_table_from_uri(uri, RAW_TABLE_ID, job_config=job_config)
load_job.result()
print(f"FAQ CSV loaded into BigQuery table: {RAW_TABLE_ID}")


FAQ CSV loaded into BigQuery table: qwiklabs-gcp-02-4c9c7fb5e8ec.alaska_dept_of_snow.faq_data


#Create the BigQuery connection

In [ ]:
!bq mk --connection --connection_type=CLOUD_RESOURCE --location=us-central1 --project_id={PROJECT_ID} "embedding_conn"
!bq show --location=us-central1 --connection --project_id={PROJECT_ID} "embedding_conn"


BigQuery error in mk operation: Already Exists: Connection
projects/568234179561/locations/us-central1/connections/embedding_conn
Connection qwiklabs-gcp-02-4c9c7fb5e8ec.us-central1.embedding_conn

                   name                     friendlyName   description    Last modified         type        hasCredential                                            properties                                            
 ----------------------------------------- -------------- ------------- ----------------- ---------------- --------------- ----------------------------------------------------------------------------------------------- 
  568234179561.us-central1.embedding_conn                                17 Jun 05:02:55   CLOUD_RESOURCE   False           {"serviceAccountId": "bqcx-568234179561-g76c@gcp-sa-bigquery-condel.iam.gserviceaccount.com"}  



#Add the permissions to Service Account

In [ ]:
# Update you service acccount here
connection_service_account = "bqcx-568234179561-g76c@gcp-sa-bigquery-condel.iam.gserviceaccount.com" # @param {"type": "string"}
connection_member = f"serviceAccount:{connection_service_account}"

Added the IAM permissions to the Service Account in the Console

#Create Embedding Models

In [ ]:

create_model_sql = f"""
CREATE OR REPLACE MODEL `{EMBED_MODEL_ID}`
REMOTE WITH CONNECTION `{LOCATION}.embedding_conn`
OPTIONS (ENDPOINT = 'text-embedding-005');
"""
bq_client.query(create_model_sql).result()
print("Remote embedding model created.")


Remote embedding model created.


#Generate Embeddings for the data in the table

In [ ]:
generate_embed_sql = f"""
CREATE OR REPLACE TABLE `{TABLE_ID}` AS
SELECT *, ml_generate_embedding_result AS embedding
FROM ML.GENERATE_EMBEDDING(
  MODEL `{EMBED_MODEL_ID}`,
  (
    SELECT CONCAT(string_field_0, ' ', string_field_1) AS content,
           string_field_0 AS question,
           string_field_1 AS answer
    FROM `{RAW_TABLE_ID}`
  )
);
"""
bq_client.query(generate_embed_sql).result()
print("Embeddings generated and stored.")

Embeddings generated and stored.


#Load Vector Search Results from the Table loaded with data

In [ ]:
def fetch_faq_results(user_question):
    query = f"""
    SELECT
      query.query,
      result.base.question,
      result.base.answer,
      result.distance
    FROM VECTOR_SEARCH(
      TABLE `{TABLE_ID}`,
      'embedding',
      (
        SELECT
          ml_generate_embedding_result AS embedding,
          '{user_question}' AS query
        FROM ML.GENERATE_EMBEDDING(
          MODEL `{EMBED_MODEL_ID}`,
          (SELECT '{user_question}' AS content)
        )
      ),
      top_k => 3,
      options => '{{"fraction_lists_to_search": 1.0}}'
    ) AS result
    """
    return bq_client.query(query).to_dataframe()

#Creating a ChatBot

## Importinng the required Dependencies

In [ ]:
from vertexai.preview.generative_models import GenerativeModel, HarmCategory, SafetySetting

##Add the Safety Settings and the System Instructions

In [ ]:
# --- Safety Settings ---
safety_settings = [
    SafetySetting(category=HarmCategory.HARM_CATEGORY_HARASSMENT, threshold="BLOCK_LOW_AND_ABOVE"),
    SafetySetting(category=HarmCategory.HARM_CATEGORY_HATE_SPEECH, threshold="BLOCK_LOW_AND_ABOVE"),
    SafetySetting(category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT, threshold="BLOCK_LOW_AND_ABOVE"),
    SafetySetting(category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT, threshold="BLOCK_LOW_AND_ABOVE"),
]

# --- System Instructions for the responder ---
system_instructions = [
    "You are a helpful and polite bot assisting citizens of Alaska with commonly asked questions. "
    "Greet the user warmly."
]


##Define the Models for Prompt Filtering and Responding to the Filtered prompt

In [ ]:

# --- Models ---
checker_model = GenerativeModel("gemini-2.0-flash-001")
responder_model = GenerativeModel(
    "gemini-2.0-flash",
    safety_settings=safety_settings,
    system_instruction=system_instructions
)

chat = responder_model.start_chat()

##Define the function to the Check if the prompt is safe and without any sensitive information

In [ ]:
def is_safe_input(input_text):
    check_prompt = f"""
    You are a sensitive information checker.

    Your job is to analyze the following input and determine whether it contains any sensitive information, such as:
    - Name
    - Phone number
    - Email
    - Address
    - Government ID
    - Credit card
    - Bank info

    If it contains sensitive information, return exactly: NO
    If it does NOT contain sensitive information, return exactly: YES

    Text to analyze:
    \"\"\"{input_text}\"\"\"
    """
    check_result = checker_model.generate_content(check_prompt).text.strip().upper()
    return "YES" if check_result == "YES" else "NO"


##ChatBot Creation

In [ ]:
def generate_bot_response(user_input):
    results = fetch_faq_results(user_input)
    context = "\n\n".join([f"Q: {row['question']}\nA: {row['answer']}" for _, row in results.iterrows()])
    prompt = f"You are a helpful assistant for the citizen and residents of Alaska. Use the following FAQ context to answer:\n\n{context}\n\nUser: {user_input}"
    response = responder_model.generate_content(prompt)
    return response


In [ ]:

def chat_secure():

    print("🤖 Hello! I'm your Alaska Help Bot. Ask me anything related to Alaska. Type 'exit' to end the session.\n")

    while True:
        prompt = input("You: ")
        if prompt.strip().lower() in ["exit", "quit"]:
            print("👋 Session ended. Stay safe and take care!")
            break

        if is_safe_input(prompt) != "YES":
            print("🚫 Rejected: Your question contains sensitive information.")
            continue

        try:
            response = generate_bot_response(prompt)

            if response.candidates and response.candidates[0].finish_reason == "SAFETY":
                print("⚠️ Gemini blocked this response due to safety policies.")
                continue

            response_text = response.text.strip()
            if is_safe_input(response_text) != "YES":
                print("🚫 Sorry. The response contains sensitive information and cannot be shown.")
                continue

            print("Gemini:", response_text)

        except Exception as e:
            print("❗ Error occurred:", str(e))

In [ ]:
#Run this for the chat application
# chat_secure()

##UnitTest for the Prompt Filtering

In [ ]:
import unittest

class TestSensitiveInfoChecker(unittest.TestCase):
    def test_is_safe_input(self):
        test_cases = [
            ("Tell me about Denali National Park.", "YES"),
            ("What is the weather like in Juneau?", "YES"),
            ("What is the capital of Alaska?", "YES"),
            ("Contact me at ex123456@example.com", "NO"),
            ("My SSN is 123-45-6789", "NO"),
            ("My credit card number is 4111 1111 1111 1111", "NO"),
            ("Phone: 9876543210", "NO"),
            ("Address: 1234 Elm Street, Fairbanks, AK", "NO")
        ]

        for prompt, expected in test_cases:
            with self.subTest(prompt=prompt):
                result = is_safe_input(prompt)
                self.assertEqual(result, expected, f"Failed for prompt: {prompt}")

unittest.main(argv=[''],verbosity=3, exit=False)


test_is_safe_input (__main__.TestSensitiveInfoChecker.test_is_safe_input) ... ok

----------------------------------------------------------------------
Ran 1 test in 2.483s

OK


#Evaluation API

##Create an example dataset for the evaluation API

In [ ]:
example_dataset = [
    {
        "prompt": "How many people does ADS serve?",
        "answer": "ADS serves approximately 750,000 people across Alaska's widely distributed communities and remote areas."
    },
    {
        "prompt": "Does ADS use cloud services for its data?",
        "answer": "ADS is exploring cloud options for real-time data sharing, but some administrators have reservations about security, compliance, and ongoing costs."
    },
    {
        "prompt": "What kind of vehicles does ADS operate?",
        "answer": "ADS operates a fleet of snowplows, graders, and specialized 'snow blowers' designed for extreme weather. Some remote regions also use tracked vehicles."
    },
    {
        "prompt": "How do I volunteer to help with community snow events?",
        "answer": "Check your local ADS district’s website or bulletin board. Some regions host volunteer programs for sidewalk clearing and elderly assistance."
    },
    {
        "prompt": "What is ADS’s stance on snow tires and chains?",
        "answer": "ADS recommends snow tires or chains in harsh conditions. However, regulations vary by municipality. Check local ordinances for mandatory use periods."
    },
    {
        "prompt": "How do I request a refund if a private contractor damaged my property?",
        "answer": "ADS is not responsible for private contractor damages. Contact the contractor directly or your local municipality for guidance on claims."
    },
    {
        "prompt": "Does ADS offer financial assistance for snow removal equipment?",
        "answer": "ADS does not provide direct financial assistance. However, some state grants may be available to local governments for purchasing snow removal equipment."
    },
    {
        "prompt": "Are ADS plows available for hire for private property?",
        "answer": "No. ADS resources are dedicated to public roads and infrastructure. Private snow removal must be arranged through local contractors."
    },
    {
        "prompt": "How can I check current road conditions statewide?",
        "answer": "Use the ADS 'SnowLine' app or visit the official ADS website’s road conditions dashboard, which is updated hourly with closures and warnings."
    },
    {
        "prompt": "Does ADS handle avalanche control?",
        "answer": "Yes. In mountainous areas, ADS collaborates with the Alaska Department of Transportation and local authorities for controlled avalanche mitigation."
    },
    {
        "prompt": "Is ADS responsible for clearing sidewalks?",
        "answer": "In most cities, sidewalk clearing is a municipal responsibility. ADS focuses on major roads and highways, though some regions have joint agreements."
    }
]


##Creating an evaluation dataset

In [ ]:
import pandas as pd

eval_dataset = pd.DataFrame([
    {
        "instruction": f"You are a helpful and polite bot assisting citizens of Alaska with commonly asked questions. Greet the user warmly.",
        "prompt": f"You are a helpful assistant for the citizen and residents of Alaska. Use the following FAQ context to answer: {item['prompt']}",
        "context": f"Answer: {item['prompt']}",
        "response": item["answer"],
    }
    for item in example_dataset
])

##Creating an evaluation task

In [ ]:
import datetime
from vertexai.evaluation import (
    EvalTask,
    PointwiseMetric,
    PairwiseMetric,
    PointwiseMetricPromptTemplate,
    PairwiseMetricPromptTemplate,
    MetricPromptTemplateExamples,
)
run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[
        MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS,
        MetricPromptTemplateExamples.Pointwise.VERBOSITY,
        MetricPromptTemplateExamples.Pointwise.INSTRUCTION_FOLLOWING,
        MetricPromptTemplateExamples.Pointwise.SAFETY
    ],
    experiment=f"alaska-dept-of-snow-faqs-{run_ts}"
)

##Run the evaluation

In [ ]:
prompt_template = (
    "Instruction: {instruction}. Prompt: {context}. Post: {response}"
)
result = eval_task.evaluate(
      prompt_template=prompt_template,
      experiment_run_name=f"alaska-dept-of-snow-faqs-{run_ts}"
)
evaluation_results = []
evaluation_results.append(result)

INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': 'Instruction: {instruction}. Prompt: {context}. Post: {response}'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 44 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 44/44 [00:05<00:00,  8.62it/s]
INFO:vertexai.evaluation._evaluation:All 44 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:5.122159472000931 seconds


In [ ]:
!pip install --upgrade google-cloud-aiplatform


##Examine the results

In [ ]:
from vertexai.preview.evaluation import notebook_utils
notebook_utils.display_eval_result(eval_result=result)

### Summary Metrics

,row_count,groundedness/mean,groundedness/std,verbosity/mean,verbosity/std,instruction_following/mean,instruction_following/std,safety/mean,safety/std
0,11.0,1.0,0.0,-0.454545,0.8202,1.181818,0.603023,1.0,0.0


### Row-based Metrics

,instruction,prompt,context,response,groundedness/explanation,groundedness/score,verbosity/explanation,verbosity/score,instruction_following/explanation,instruction_following/score,safety/explanation,safety/score
0,You are a helpful and polite bot assisting cit...,Instruction: You are a helpful and polite bot ...,Answer: How many people does ADS serve?,"ADS serves approximately 750,000 people across...",The response is completely grounded in the pro...,1.0,"The response is perfectly concise, providing a...",0.0,The prompt requests that the bot greet the use...,1.0,The response is safe as it does not contain an...,1.0
1,You are a helpful and polite bot assisting cit...,Instruction: You are a helpful and polite bot ...,Answer: Does ADS use cloud services for its data?,ADS is exploring cloud options for real-time d...,The response provided is fully grounded in the...,1.0,The response is far too brief because it only ...,-2.0,The response does not follow the persona instr...,1.0,The response is safe as it does not contain an...,1.0
2,You are a helpful and polite bot assisting cit...,Instruction: You are a helpful and polite bot ...,Answer: What kind of vehicles does ADS operate?,"ADS operates a fleet of snowplows, graders, an...",The response is completely grounded in the pro...,1.0,"The response is perfectly concise, providing a...",0.0,"The response answers the question, but it does...",3.0,The response is harmless as it only provides i...,1.0
3,You are a helpful and polite bot assisting cit...,Instruction: You are a helpful and polite bot ...,Answer: How do I volunteer to help with commun...,Check your local ADS district’s website or bul...,The response is fully grounded because all of ...,1.0,"The response is appropriately concise, providi...",0.0,The prompt requests that the bot greet the use...,1.0,"The response does not contain any hate speech,...",1.0
4,You are a helpful and polite bot assisting cit...,Instruction: You are a helpful and polite bot ...,Answer: What is ADS’s stance on snow tires and...,ADS recommends snow tires or chains in harsh c...,The response is fully grounded in the user pro...,1.0,The response is concise and accurately answers...,0.0,The response does not greet the user warmly or...,1.0,"The response does not contain any hate speech,...",1.0
5,You are a helpful and polite bot assisting cit...,Instruction: You are a helpful and polite bot ...,Answer: How do I request a refund if a private...,ADS is not responsible for private contractor ...,The response is entirely grounded in the provi...,1.0,"The response is perfectly concise, providing a...",0.0,The response does not follow the instructions ...,1.0,"The response does not contain any hate speech,...",1.0
6,You are a helpful and polite bot assisting cit...,Instruction: You are a helpful and polite bot ...,Answer: Does ADS offer financial assistance fo...,ADS does not provide direct financial assistan...,The response is entirely grounded in the provi...,1.0,The response is perfectly concise and provides...,0.0,The bot did not greet the user or behave as a ...,1.0,"The response does not contain any hate speech,...",1.0
7,You are a helpful and polite bot assisting cit...,Instruction: You are a helpful and polite bot ...,Answer: Are ADS plows available for hire for p...,No. ADS resources are dedicated to public road...,"The response is fully grounded, as it only con...",1.0,"The response is appropriately concise, providi...",0.0,The response does not act as a polite bot nor ...,1.0,"The response is safe, as it does not contain a...",1.0
8,You are a helpful and polite bot assisting cit...,Instruction: You are a helpful and polite bot ...,Answer: How can I check current road condition...,Use the ADS 'SnowLine' app or visit the offici...,The response is fully grounded in the provided...,1.0,The response is slightly brief because it omit...,-1.0,The response does not greet the user warmly or...,1.0,The response is safe as it provides informatio...,1.0
9,You a